In [1]:
import torch
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
from torch.nn.functional import relu, elu, relu6, sigmoid, tanh, softmax
from torch.nn import Linear, Conv2d, BatchNorm2d, MaxPool2d, Dropout2d

import time
from datetime import datetime, timedelta
import matplotlib.dates as md



%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import networkx as nx
import os

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [2]:
df_place_details = pd.read_csv("./data/place_details.csv")
df_edges = pd.read_csv('data/graph_edges_with_directionality - Kopie.csv')

In [3]:
G = nx.DiGraph()
for orig,desti in zip(df_edges['origin_place_id'], df_edges['destination_place_id']):
    G.add_edge(orig, desti)
nodes = list(G.nodes())

In [4]:
dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

In [5]:
filename = "./gen_data_frame_h5/df_our_sample_nans.h5"
df = pd.read_hdf(filename,parse_dates=['start_interval_s'], date_parser=dateparse, index_col = 1)

In [6]:
idx = df.index

In [7]:
df = df.drop(idx[idx.year == 2013])

In [8]:
idx = df.index
df = df.drop(idx[idx.hour < 5])

In [9]:
idx = df.index
df = df.drop(idx[idx.hour > 21])
idx = df.index

In [11]:
for n in nodes:
    df[n] = df[n]/162

In [12]:
dateparse_day = lambda x: pd.datetime.strptime(x, '%Y-%m-%d')

In [13]:
idx[0], idx[-1]

(Timestamp('2015-01-01 05:00:00'), Timestamp('2015-06-29 21:55:00'))

In [14]:
idx[0].date() == idx[1].date()

True

In [15]:
(idx[-1].date() - idx[0].date()).days

179

In [16]:
idx_dates = np.array([i.date() for i in idx])

In [17]:
sums = []
for i in range((idx[-1].date() - idx[0].date()).days):
    check_date = (idx[0] + timedelta(days = i)).date()
    match_date = idx_dates == check_date
    sums.append(sum(match_date))

In [18]:
max_date = (idx[0] + timedelta(days = int(np.argmax(sums)))).date()
max_matches = idx_dates == max_date
max_times = np.array([(np.datetime64(idx[i]) - np.datetime64(idx[i], 'D'))/np.timedelta64(1, 'D') for i in np.arange(len(max_matches))[max_matches]])

In [19]:
i = 0
check_date = (idx[0] + timedelta(days = i)).date()
match_date = idx_dates == check_date

In [20]:
times = np.array([(np.datetime64(idx[i]) - np.datetime64(idx[i], 'D'))/np.timedelta64(1, 'D') for i in np.arange(len(max_matches))[match_date]])
nan_times = np.array([t if t in times else np.nan for t in max_times])

vels = np.array(df[nodes[0]])[match_date]
nan_vels = np.array([v if t == t else np.nan for v,t in zip(vels,nan_times)])

In [21]:
def nan_helper(y):
    return np.isnan(y), lambda z: z.nonzero()[0]

# Interpolate missing values

In [22]:
for i,node in list(enumerate(nodes)):
    print((i,node))
    completed_vels = []
    check_vels = []
    for i in range((idx[-1].date() - idx[0].date()).days + 1):
        check_date = (idx[0] + timedelta(days = i)).date()
        match_date = idx_dates == check_date
        vels = np.array(df[node])[match_date]
        check_vels.extend(vels)
        nans, x = nan_helper(vels)
        try:
            vels[nans] = np.interp(x(nans), x(~nans), vels[~nans])
        except ValueError:
            vels[nans] = np.nanmean(df[node])
        completed_vels.extend(vels)
    df[node] = completed_vels

(0, 'ChIJDfnLZ1lSUkYRchF_if80LfE')
(1, 'ChIJsRh_oFlSUkYRkBBHQUeoKcI')
(2, 'ChIJ9cuitllSUkYRd6ZoiDSinIM')
(3, 'ChIJk2Y2M1hSUkYR8cXNNRfosIY')
(4, 'ChIJCy4tOFhSUkYRBiS5iJUrGnY')
(5, 'ChIJj1RhMlhSUkYRxwx00g4P0QE')
(6, 'ChIJuYkcKlhSUkYRFPCipW5rTvU')
(7, 'ChIJP6TdhFdSUkYRdrsWKXZMAs8')
(8, 'ChIJf9Y9sFdSUkYRmaDEJhCweGc')
(9, 'ChIJozaGTFZSUkYRNtWl2AGUPkI')
(10, 'ChIJ4QuVTlZSUkYRRDRPcHdYULQ')
(11, 'ChIJozaGTFZSUkYRjY7TpiWcsro')
(12, 'ChIJr8hSvVdSUkYR9El_JWHKncY')
(13, 'ChIJuYkcKlhSUkYRkyJyE8H3_fI')
(14, 'ChIJV02yLVhSUkYR5Urrpdyav5w')
(15, 'ChIJI3hCtFlSUkYRPnfwEKyXdTk')
(16, 'ChIJOW51C1lSUkYRRv6n34W9-Zg')
(17, 'ChIJj5kfbFlSUkYRcDRZTsny7_c')
(18, 'ChIJpyBCaFlSUkYR3LHwM6zxRjU')
(19, 'ChIJL3cdF1dSUkYRrSCPkRNdtPE')
(20, 'ChIJgXdTbFdSUkYR33h2wlG_Zwk')
(21, 'ChIJ64wzbldSUkYR5FeQ9EKDTkU')
(22, 'ChIJ7feGfFdSUkYRmmsyva5Yh7g')
(23, 'ChIJRwYQ1llSUkYRMKaZjz2gpdg')
(24, 'ChIJmQYH01lSUkYRA5sfbpFn4ag')
(25, 'ChIJaU2CalhSUkYR16Poj9gzgIw')
(26, 'ChIJt2H3cFhSUkYRAkkFfX87Bq8')
(27, 'ChIJRZDreVhSUkYRo3ujslhgkNQ')
(2

In [23]:
completed_vels = np.array(completed_vels)
check_vels = np.array(check_vels)

In [28]:
def nan_helper(y):
    return np.isnan(y), lambda z: z.nonzero()[0]

In [29]:
y = np.array([np.nan,5, np.nan, 2, np.nan, np.nan, 2, 2, np.nan, 0])

nans, x= nan_helper(y)
y[nans]= np.interp(x(nans), x(~nans), y[~nans])

In [30]:
print(y.round(2))

[5.  5.  3.5 2.  2.  2.  2.  2.  1.  0. ]


In [31]:
idx = df.index
df = df.drop(idx[idx.weekday > 4])
idx = df.index

In [32]:
len(df[nodes[0]])

26112

In [33]:
df.to_hdf('df_2015_weekdays_6_to_21_completed.h5', key='df', mode='w')

In [34]:
df

,ChIJDfnLZ1lSUkYRchF_if80LfE,ChIJsRh_oFlSUkYRkBBHQUeoKcI,ChIJ9cuitllSUkYRd6ZoiDSinIM,ChIJk2Y2M1hSUkYR8cXNNRfosIY,ChIJCy4tOFhSUkYRBiS5iJUrGnY,ChIJj1RhMlhSUkYRxwx00g4P0QE,ChIJuYkcKlhSUkYRFPCipW5rTvU,ChIJP6TdhFdSUkYRdrsWKXZMAs8,ChIJf9Y9sFdSUkYRmaDEJhCweGc,ChIJozaGTFZSUkYRNtWl2AGUPkI,...,ChIJ2ejtT_FSUkYRtmPci5MfBGk,ChIJw6JrT_FSUkYRWsJ_wS11MGA,ChIJX_J3OPFSUkYR8N1MgR4DmCE,ChIJP_V5O_FSUkYRIdmGiDO_euc,ChIJ5z1yIfFSUkYRYsAwlasCBM4,ChIJjz-_3_BSUkYRRk9hBB16k-Q,ChIJu3Ts5_BSUkYRPEnUSqJE1-4,ChIJY9y76_BSUkYRUTObHO7pJvE,ChIJm_l8k_BSUkYRIRwmcpaM9XU,ChIJr2_smvBSUkYR_7t4J6k2Azg
start_interval_s,,,,,,,,,,,,,,,,,,,,,
2015-01-01 05:00:00,0.111140,0.172750,0.173990,0.127037,0.184416,0.096606,0.157365,0.190361,0.181144,0.148066,...,0.169326,0.149589,0.152412,0.173856,0.134884,0.111483,0.086569,0.110170,0.133484,0.110598
2015-01-01 05:05:00,0.107019,0.219226,0.209875,0.127603,0.190691,0.128892,0.185761,0.206296,0.215152,0.195899,...,0.200911,0.142510,0.160765,0.179641,0.164971,0.146860,0.115411,0.111054,0.109098,0.105185
2015-01-01 05:10:00,0.096177,0.181779,0.213290,0.137347,0.171767,0.147660,0.195988,0.220516,0.263330,0.257344,...,0.209405,0.147751,0.154212,0.153138,0.182234,0.176258,0.148260,0.161860,0.150212,0.135538
2015-01-01 05:15:00,0.071919,0.092941,0.168872,0.154634,0.136262,0.174596,0.193494,0.197691,0.300248,0.334613,...,0.196371,0.157890,0.135294,0.136101,0.170361,0.175254,0.152594,0.178801,0.169661,0.140574
2015-01-01 05:20:00,0.049534,0.039543,0.130033,0.171893,0.101630,0.191519,0.190993,0.140050,0.230060,0.351313,...,0.172473,0.164723,0.121971,0.128048,0.148060,0.157258,0.135209,0.171991,0.166543,0.127569
2015-01-01 05:25:00,0.087423,0.059363,0.150252,0.181332,0.122701,0.199059,0.195635,0.147455,0.176581,0.279852,...,0.164170,0.160725,0.133357,0.139243,0.145376,0.138428,0.120255,0.165736,0.168865,0.128213
2015-01-01 05:30:00,0.141739,0.104062,0.186101,0.147017,0.187124,0.203415,0.198622,0.164502,0.173626,0.251163,...,0.170630,0.153948,0.168469,0.173805,0.172467,0.155725,0.136229,0.173453,0.181079,0.143513
2015-01-01 05:35:00,0.173814,0.136488,0.207264,0.121391,0.231919,0.206883,0.202556,0.180066,0.193218,0.251834,...,0.180797,0.162304,0.196182,0.201309,0.210434,0.205077,0.171154,0.192003,0.185629,0.187367
2015-01-01 05:40:00,0.200679,0.152357,0.208096,0.133951,0.267791,0.204979,0.204232,0.188227,0.209249,0.252934,...,0.183335,0.174758,0.200781,0.196898,0.219831,0.228210,0.191675,0.205071,0.174660,0.219689
